In [1]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine

In [6]:
df = pd.read_csv (r'AccidentesAviones.csv')
df.head(5)

,Unnamed: 0,fecha,HORA declarada,Ruta,OperadOR,flight_no,route,ac_type,registration,cn_ln,all_aboard,PASAJEROS A BORDO,crew_aboard,cantidad de fallecidos,passenger_fatalities,crew_fatalities,ground,summary
0,0,"September 17, 1908",1718,"Fort Myer, Virginia",Military - U.S. Army,?,Demonstration,Wright Flyer III,?,1,2,1,1,1,1,0,0,"During a demonstration flight, a U.S. Army fly..."
1,1,"September 07, 1909",?,"Juvisy-sur-Orge, France",?,?,Air show,Wright Byplane,SC1,?,1,0,1,1,0,0,0,Eugene Lefebvre was the first pilot to ever be...
2,2,"July 12, 1912",0630,"Atlantic City, New Jersey",Military - U.S. Navy,?,Test flight,Dirigible,?,?,5,0,5,5,0,5,0,First U.S. dirigible Akron exploded just offsh...
3,3,"August 06, 1913",?,"Victoria, British Columbia, Canada",Private,?,?,Curtiss seaplane,?,?,1,0,1,1,0,1,0,The first fatal airplane accident in Canada oc...
4,4,"September 09, 1913",1830,Over the North Sea,Military - German Navy,?,?,Zeppelin L-1 (airship),?,?,20,?,?,14,?,?,0,The airship flew into a thunderstorm and encou...


In [7]:
df.columns

Index(['Unnamed: 0', 'fecha', 'HORA declarada', 'Ruta', 'OperadOR',
       'flight_no', 'route', 'ac_type', 'registration', 'cn_ln', 'all_aboard',
       'PASAJEROS A BORDO', 'crew_aboard', 'cantidad de fallecidos',
       'passenger_fatalities', 'crew_fatalities', 'ground', 'summary'],
      dtype='object')

In [8]:
df = df.rename(columns= {'Unnamed: 0' : 'id_vuelo' , 
                        'HORA declarada' : 'hora' , 
                        'Ruta' : 'ruta' ,
                        'OperadOR' : 'operador' ,
                        'flight_no' : 'num_vuelo' , 
                        'route' : 'ruta' ,
                        'PASAJEROS A BORDO' : 'pass_aboard' ,
                        'passenger_fatalities' : 'pass_fatalities' ,
                        'cantidad de fallecidos' : 'total_fatalities'})

In [9]:
df.fecha = pd.to_datetime(df.fecha)
df.fecha = pd.to_datetime (df.fecha , format = "%d/%m/%Y")

In [10]:
df['hora'] = df['hora'].replace('?' , '00:00')
df['hora'] = df['hora'].replace('c 18:00' , '18:00')

In [11]:
def normalize_hour(value):
    if ':' not in value:
        return f'{value[:2]}:{value[-2:]}'
    else:
        return value
df['hora'] = df['hora'].apply(lambda x: normalize_hour(x))

In [12]:
df = df.replace({'?' : 'Sin Dato'})

In [13]:
df['hora'] = df.hora.str.slice(-5)

In [14]:
df['hora'] = pd.to_datetime(df.hora , errors = 'ignore')

In [10]:
def conexion_database(nombre_base):
    cadena_conexion= f"mysql+pymysql://root:@127.0.0.1:3306/{nombre_base}"
    conexion= create_engine(cadena_conexion)
    return conexion

In [7]:
def crear_base_de_datos():
    print('Ingrese nombre de base de datos: ')
    conexion = pymysql.connect(host="localhost",
                            user = "root",
                            password=''
                            )
    cursor = conexion.cursor()
    nombre_base = input()
    query =f"CREATE DATABASE IF NOT EXISTS {nombre_base};"
    cursor.execute(query)
    conexion_database(nombre_base)
    return nombre_base

In [5]:
def importar_tablas(df , nombre_base):
    conexion = conexion_database(nombre_base)
    nombre = input('Ingrese nombre de la tabla: ')
    df.to_sql(name=nombre , con=conexion , index= False , if_exists='append')

In [ ]:
conexion_database('accidentes_aviones')
crear_base_de_datos()

In [12]:
df2 = pd.read_csv(r'airline-safety\airline-safety.csv')

In [13]:
importar_tablas(df2, 'accidentes_aviones')